# 1. Setup

In [2]:
%load_ext watermark

import glob
import datetime
import numpy as np
import pandas as pd
import xarray as xr

%watermark -iv -co -v

Python implementation: CPython
Python version       : 3.12.10
IPython version      : 9.2.0

conda environment: cpl_ppe_co2

numpy : 2.0.1
xarray: 2025.4.0
pandas: 2.2.2
sys   : 3.12.10 | packaged by conda-forge | (main, Apr 10 2025, 22:21:13) [GCC 13.3.0]



# Load crosswalk

In [19]:
pdir = '/glade/u/home/bbuchovecky/projects/cpl_ppe_co2/hist/data/paramfiles'
crosswalk = pd.read_csv("coupPPE-hist_crosswalk_edited.csv", index_col=0, dtype=str)

pert_param = set(crosswalk[(crosswalk['loc'] == 'P') | (crosswalk['loc'] == 'N')].param.unique())

In [34]:
pdef = xr.open_dataset("~/projects/cpl_ppe_co2/hist/data/paramfiles/coupPPE-hist.000.nc", decode_timedelta=False)

In [36]:
pdef["vcmaxse_sf"].values

array(1.)

In [37]:
pdef["jmaxse_sf"].values

array(1.)

In [38]:
pdef["tpuse_sf"].values

array(1.)

# Compare parameter files from CMIP6 and Claire's PPE

In [20]:
cmip_bhist = xr.open_dataset('/glade/p/cesmdata/cseg/inputdata/lnd/clm2/paramdata/clm5_params.c171117.nc', decode_timedelta=False)
cesm_ppe = xr.open_dataset('/glade/u/home/czarakas/coupled_PPE/data/paramfiles/OAAT0000.nc', decode_timedelta=False)

cmip_bhist_param = set(cmip_bhist.data_vars)
cesm_ppe_param = set(cesm_ppe.data_vars)

In [27]:
# cesm_ppe_param - cmip_bhist_param

In [29]:
print("perturbed parameters not in the PPE CLM version parameter file")
print(pert_param - cesm_ppe_param)

perturbed parameters not in the PPE CLM version parameter file
{'maximum_leaf_wetted_fraction'}


In [30]:
print("perturbed parameters not in the CMIP6 CLM version parameter file")
print(pert_param - cmip_bhist_param)

perturbed parameters not in the CMIP6 CLM version parameter file
{'vcmaxse_sf', 'jmaxse_sf', 'd_max', 'fff', 'tpuse_sf', 'maximum_leaf_wetted_fraction', 'lmrha', 'jmaxb0', 'cv'}


In [4]:
pftname = np.array([str(s.strip())[2:-1] for s in cesm_ppe.pftname.values])

In [ ]:
disagree_dict = dict()

for param in sorted(cesm_ppe_param & cmip_bhist_param):
    ispft = False
    agreement = min(cmip_bhist[param].values.flatten() == cesm_ppe[param].values.flatten())
    if 'pft' in cesm_ppe[param].dims:
        ispft = True
        print(param, 'pft', agreement)
    else:
        print(param, agreement)
    
    if not agreement:
        print(' CESM PPE\n', cesm_ppe[param].values.flatten())
        print(' CMIP6   \n', cmip_bhist[param].values.flatten())
        disagree_dict[param] = (ispft, cesm_ppe[param].values, cmip_bhist[param].values)

FUN_fracfixers pft False
 CESM PPE
 [0.   0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25
 0.25 0.   0.   0.   0.   0.   0.   0.   0.   1.   1.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.25 0.25 0.25 0.25 0.   0.   1.   1.  ]
 CMIP6   
 [0.   0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25
 0.25 0.   0.   0.   0.   0.   0.   0.   0.   1.   1.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   1.   1.  ]
a_fix pft True
aereoxid True
akc_active pft False
 CESM PPE
 [0.    0.06  0.06  0.06  0.06  0.06  0.006 0.06  0.06  0.06  0.06  0.06
 0.06  0.06  0.6   0.06  0.06  0

In [18]:
print('parameters that are not the same between the two parameter files:')
print('---')
print(np.array(list(disagree_dict.keys())))

parameters that are not the same between the two parameter files:
---
['FUN_fracfixers' 'akc_active' 'akn_active' 'allconsl' 'allconss' 'arootf'
 'arooti' 'astemf' 'baset' 'bfact' 'c3psn' 'declfact' 'ekc_active'
 'ekn_active' 'ffrootcn' 'fleafi' 'flnr' 'froot_leaf' 'frootcn_max'
 'frootcn_min' 'fsr_pft' 'fstemcn' 'fun_cn_flex_c' 'gddmin' 'graincn'
 'grnfill' 'hybgdd' 'i_flnr' 'i_vcad' 'kc_nonmyc' 'kmax' 'kn_nonmyc'
 'krmax' 'laimx' 'leafcn' 'leafcn_max' 'leafcn_min' 'lfemerg'
 'livewdcn_max' 'livewdcn_min' 'max_NH_planting_date'
 'max_SH_planting_date' 'mbbopt' 'medlynslope' 'mergetoclmpft'
 'min_NH_planting_date' 'min_SH_planting_date' 'min_planting_temp' 'mxmat'
 'mxtmp' 'pconv' 'pftpar20' 'pftpar28' 'pftpar29' 'pftpar31'
 'planting_temp' 'pprodharv10' 'rholnir' 'rholvis' 'rhosnir' 'rhosvis'
 'root_dmx' 's_flnr' 's_vcad' 'slatop' 'taulnir' 'taulvis' 'xl' 'ztopmx']


In [38]:
print('total pfts:', len(pftname))
print('----------------------------------')
print('number of different pft parameters')
print('----------------------------------')
for key, value in disagree_dict.items():
    if value[0] and len(value[1].shape) == 1:
        print(key, len(pftname[value[1] != value[2]]))
        # print(pftname[value[1] != value[2]])
        # print(value[1] - value[2])

total pfts: 79
----------------------------------
number of different pft parameters
----------------------------------
FUN_fracfixers 4
akc_active 4
akn_active 4
allconsl 53
allconss 53
arootf 67
arooti 53
astemf 59
baset 4
bfact 17
c3psn 4
declfact 17
ekc_active 4
ekn_active 4
ffrootcn 4
fleafi 53
flnr 4
froot_leaf 2
frootcn_max 4
frootcn_min 4
fsr_pft 76
fstemcn 4
fun_cn_flex_c 4
gddmin 17
graincn 17
grnfill 53
hybgdd 53
i_flnr 4
i_vcad 4
kc_nonmyc 2
kn_nonmyc 2
krmax 15
laimx 53
leafcn 2
leafcn_max 2
leafcn_min 2
lfemerg 53
livewdcn_max 4
livewdcn_min 4
max_NH_planting_date 55
max_SH_planting_date 53
mbbopt 20
medlynslope 5
mergetoclmpft 5
min_NH_planting_date 53
min_SH_planting_date 53
min_planting_temp 53
mxmat 53
mxtmp 53
pconv 63
pftpar20 1
pftpar28 67
pftpar29 70
pftpar31 75
planting_temp 59
pprodharv10 71
rholnir 78
rholvis 78
rhosnir 11
rhosvis 11
root_dmx 4
s_flnr 4
s_vcad 2
slatop 3
taulnir 78
taulvis 11
xl 43
ztopmx 53


In [19]:
# Parameters to perturb
param_list = [
    'kmax',
    'KCN',
    'sand_pf',
]

# Where to look for parameter ranges csv file
indir = '/glade/u/home/linnia/ctsm6_ppe/gen_ensembles/gen_paramfiles'

# Where to save the parameter files
outdir = '/glade/u/home/bbuchovecky/projects/cpl_ppe_co2/hist/data/paramfiles'

# Load the parameter ranges
csv = 'ctsm6lhc_paramranges_11262024.csv'
params = pd.read_csv(f'{indir}/{csv}')
params = params.loc[params['param'].isin(param_list)]
if len(params) != len(param_list):
    print(f'Not all parameters found in the csv file, missing {set(param_list) - set(params["param"])}')
    # raise ValueError(f'Not all parameters found in the csv file, missing {set(param_list) - set(params["param"].values)}')

# Load the default parameter file
basefile = '/glade/work/linnia/CLM6-PPE/ctsm6_oaat/paramfiles/oaat0000.nc'
def_paramfile = xr.open_dataset(basefile)

Not all parameters found in the csv file, missing {'sand_pf'}
